https://livy.incubator.apache.org/examples/

In [1]:
import json, pprint, requests, textwrap

In [2]:
host = 'http://spark-livy:8998'
data = {'kind': 'pyspark'}
headers = {'Content-Type': 'application/json'}
r = requests.post(host + '/sessions', data=json.dumps(data), headers=headers)
r.json()

{'id': 2,
 'name': None,
 'appId': None,
 'owner': None,
 'proxyUser': None,
 'state': 'starting',
 'kind': 'pyspark',
 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None},
 'log': ['stdout: ', '\nstderr: ']}

In [3]:
session_url = host + r.headers['location']
r = requests.get(session_url, headers=headers)
r.json()

{'id': 2,
 'name': None,
 'appId': None,
 'owner': None,
 'proxyUser': None,
 'state': 'starting',
 'kind': 'pyspark',
 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None},
 'log': ['stdout: ',
  '\nstderr: ',
  '22/04/02 10:25:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable',
  'log4j:WARN No appenders could be found for logger (io.netty.util.internal.logging.InternalLoggerFactory).',
  'log4j:WARN Please initialize the log4j system properly.',
  'log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.']}

In [4]:
session_url

'http://spark-livy:8998/sessions/2'

In [5]:
statements_url = session_url + '/statements'
data = {'code': '1 + 1'}
r = requests.post(statements_url, data=json.dumps(data), headers=headers)
r.json()

'java.lang.IllegalStateException: Session is in state starting'

In [6]:
r.headers

{'Date': 'Sat, 02 Apr 2022 10:24:51 GMT', 'Content-Type': 'application/json;charset=utf-8', 'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked', 'Server': 'Jetty(9.3.24.v20180605)'}

In [6]:
statements_url

'http://spark-livy:8998/sessions/2/statements'

In [7]:
r = requests.get(statements_url, headers=headers)
pprint.pprint(r.json())

{'statements': [], 'total_statements': 0}


In [8]:
data = {
  'code': textwrap.dedent("""

    # Create RDD from parallelize    
    dataList = [("Java", 20000), ("Python", 100000), ("Scala", 3000)]
    rdd=spark.sparkContext.parallelize(dataList)
    """)
}

In [9]:
r = requests.post(statements_url, data=json.dumps(data), headers=headers)
pprint.pprint(r.json())

{'code': '\n'
         '\n'
         '# Create RDD from parallelize    \n'
         'dataList = [("Java", 20000), ("Python", 100000), ("Scala", 3000)]\n'
         'rdd=spark.sparkContext.parallelize(dataList)\n',
 'completed': 0,
 'id': 0,
 'output': None,
 'progress': 0.0,
 'started': 0,
 'state': 'waiting'}


In [10]:
r = requests.get(statements_url, headers=headers)
pprint.pprint(r.json())

{'statements': [{'code': '\n'
                         '\n'
                         '# Create RDD from parallelize    \n'
                         'dataList = [("Java", 20000), ("Python", 100000), '
                         '("Scala", 3000)]\n'
                         'rdd=spark.sparkContext.parallelize(dataList)\n',
                 'completed': 1648895197297,
                 'id': 0,
                 'output': {'data': {'text/plain': ''},
                            'execution_count': 0,
                            'status': 'ok'},
                 'progress': 1.0,
                 'started': 1648895196649,
                 'state': 'available'}],
 'total_statements': 1}


In [13]:
requests.delete(session_url, headers=headers)

<Response [200]>

In [12]:
session_url = 'http://spark-livy:8998/sessions/4'